In [ ]:
import numpy as np
import pandas as pd
import torch as ch
from os import path
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score, precision_score, recall_score, \
    multilabel_confusion_matrix
from sklearn.utils import resample

In [ ]:
train_data_path_3m_mrs02 = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/data_splits/3m_mrs/train_data_splits_3M_mRS_0-2_ts0.8_rs42_ns5.pth'
test_data_path_3m_mrs02 = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/data_splits/3m_mrs/test_data_3M_mRS_0-2_ts0.8_rs42.pth'

train_data_path_3m_death = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/data_splits/3m_death/train_data_splits_3M_Death_ts0.8_rs42_ns5.pth'
test_data_path_3m_death = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/data_splits/3m_death/test_data_3M_Death_ts0.8_rs42.pth'

train_data_path_hospital_death = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/data_splits/death_in_hospital/train_data_splits_Death_in_hospital_ts0.8_rs42_ns5.pth'
test_data_path_hospital_death = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/data_splits/death_in_hospital/test_data_Death_in_hospital_ts0.8_rs42.pth'

In [ ]:
# feature_order = pd.read_excel(path.join(path.dirname(path.dirname(__file__)),'LSTM', 'training', 'lstm_feature_order.xlsx'), header=None)
feature_order = pd.read_excel(path.join(path.dirname(path.abspath("")),'LSTM', 'training', 'lstm_feature_order.xlsx'), header=None)

In [ ]:
feature_order.columns = ['feature']

In [ ]:
selected_features_3m_mrs02 = [
    "prestroke_disability_(rankin)_1.0",
    "prestroke_disability_(rankin)_2.0",
    "prestroke_disability_(rankin)_3.0",
    "prestroke_disability_(rankin)_4.0",
    "prestroke_disability_(rankin)_5.0",
    'age',
    "max_NIHSS",
    "proteine C-reactive",
    "categorical_onset_to_admission_time_541-1440min",
    "categorical_onset_to_admission_time_<270min",
    "categorical_onset_to_admission_time_>1440min",
    "categorical_onset_to_admission_time_onset_unknown",
    "referral_other_hospital",
    "referral_self_referral_or_gp",
    "leucocytes",
    "categorical_ivt_91-270min",
    "categorical_ivt_<90min",
    "categorical_ivt_>540min",
    "categorical_ivt_no_ivt",
    "sodium",
    "antiplatelet_drugs_yes",
    ]

selected_features_3m_death = [
    "prestroke_disability_(rankin)_1.0",
    "prestroke_disability_(rankin)_2.0",
    "prestroke_disability_(rankin)_3.0",
    "prestroke_disability_(rankin)_4.0",
    "prestroke_disability_(rankin)_5.0",
    "leucocytes",
    "uree",
    "Glasgow Coma Scale",
    "INR",
    "ALAT",
    "max_NIHSS",
    "categorical_ivt_91-270min",
    "categorical_ivt_<90min",
    "categorical_ivt_>540min",
    "categorical_ivt_no_ivt",
    "referral_other_hospital",
    "referral_self_referral_or_gp",
    "proteine C-reactive",
]

selected_features_hospital_death = [
    "leucocytes",
    "uree",
    "Glasgow Coma Scale",
    "cholesterol HDL",
   "categorical_onset_to_admission_time_541-1440min",
    "categorical_onset_to_admission_time_<270min",
    "categorical_onset_to_admission_time_>1440min",
    "categorical_onset_to_admission_time_onset_unknown",
    "antihypert._drugs_pre-stroke_yes",
    "max_NIHSS",
    "INR",
    "PTT",
    "categorical_ivt_91-270min",
    "categorical_ivt_<90min",
    "categorical_ivt_>540min",
    "categorical_ivt_no_ivt",
    ]

In [ ]:
simplified_selected_features = [
    "prestroke_disability_(rankin)_1.0",
    "prestroke_disability_(rankin)_2.0",
    "prestroke_disability_(rankin)_3.0",
    "prestroke_disability_(rankin)_4.0",
    "prestroke_disability_(rankin)_5.0",
    "max_NIHSS",
    "categorical_onset_to_admission_time_541-1440min",
    "categorical_onset_to_admission_time_<270min",
    "categorical_onset_to_admission_time_>1440min",
    "categorical_onset_to_admission_time_onset_unknown",
    ]

selected_features_3m_mrs02 = simplified_selected_features
selected_features_3m_death = simplified_selected_features
selected_features_hospital_death = simplified_selected_features

In [ ]:
# map selected feature to feature indices
selected_features_3m_mrs02_idx = [feature_order.index[feature_order['feature'] == feature].values[0] for feature in selected_features_3m_mrs02]
selected_features_3m_death_idx = [feature_order.index[feature_order['feature'] == feature].values[0] for feature in selected_features_3m_death]
selected_features_hospital_death_idx = [feature_order.index[feature_order['feature'] == feature].values[0] for feature in selected_features_hospital_death]

In [ ]:
def evaluate_model(model, X_test, y_test):
    # calculate overall model prediction
    
    # bootstrap predictions
    roc_auc_scores = []
    matthews_scores = []
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    specificity_scores = []
    neg_pred_value_scores = []

    n_iterations = 1000
    for i in range(n_iterations):
        selected_idx = resample(np.arange(X_test.shape[0]), replace=True)
        temp_X = X_test[selected_idx]
        # ground truth
        y_bs = y_test[selected_idx]
        # predictions
        y_pred_bs = model.predict_proba(temp_X)[:, 1]
        y_pred_bs_binary = model.predict(temp_X)

        # evaluate model
        roc_auc_bs = roc_auc_score(y_bs, y_pred_bs)
        roc_auc_scores.append(roc_auc_bs)
        matthews_bs = matthews_corrcoef(y_bs, y_pred_bs_binary)
        matthews_scores.append(matthews_bs)
        accuracy_bs = accuracy_score(y_bs, y_pred_bs_binary)
        accuracy_scores.append(accuracy_bs)
        precision_bs = precision_score(y_bs, y_pred_bs_binary)  # == PPV
        recall_bs = recall_score(y_bs, y_pred_bs_binary)  # == sensitivity
        precision_scores.append(precision_bs)
        recall_scores.append(recall_bs)

        mcm = multilabel_confusion_matrix(y_bs, y_pred_bs_binary)
        tn = mcm[:, 0, 0]
        tp = mcm[:, 1, 1]
        fn = mcm[:, 1, 0]
        fp = mcm[:, 0, 1]
        specificity_bs = tn / (tn + fp)
        specificity_scores.append(specificity_bs)
        neg_pred_value_bs = tn / (tn + fn)
        neg_pred_value_scores.append(neg_pred_value_bs)

    # get medians
    median_roc_auc = np.percentile(roc_auc_scores, 50)
    median_matthews = np.percentile(matthews_scores, 50)
    median_accuracy = np.percentile(accuracy_scores, 50)
    median_precision = np.percentile(precision_scores, 50)
    median_recall = np.percentile(recall_scores, 50)
    median_specificity = np.percentile(specificity_scores, 50)
    median_neg_pred_value = np.percentile(neg_pred_value_scores, 50)

    # get 95% interval
    alpha = 100 - 95
    lower_ci_roc_auc = np.percentile(roc_auc_scores, alpha / 2)
    upper_ci_roc_auc = np.percentile(roc_auc_scores, 100 - alpha / 2)
    lower_ci_matthews = np.percentile(matthews_scores, alpha / 2)
    upper_ci_matthews = np.percentile(matthews_scores, 100 - alpha / 2)
    lower_ci_accuracy = np.percentile(accuracy_scores, alpha / 2)
    upper_ci_accuracy = np.percentile(accuracy_scores, 100 - alpha / 2)
    lower_ci_precision = np.percentile(precision_scores, alpha / 2)
    upper_ci_precision = np.percentile(precision_scores, 100 - alpha / 2)
    lower_ci_recall = np.percentile(recall_scores, alpha / 2)
    upper_ci_recall = np.percentile(recall_scores, 100 - alpha / 2)
    lower_ci_specificity = np.percentile(specificity_scores, alpha / 2)
    upper_ci_specificity = np.percentile(specificity_scores, 100 - alpha / 2)
    lower_ci_neg_pred_value = np.percentile(neg_pred_value_scores, alpha / 2)
    upper_ci_neg_pred_value = np.percentile(neg_pred_value_scores, 100 - alpha / 2)

    result_df = pd.DataFrame([{
        'auc_test': median_roc_auc,
        'auc_test_lower_ci': lower_ci_roc_auc,
        'auc_test_upper_ci': upper_ci_roc_auc,
        'matthews_test': median_matthews,
        'matthews_test_lower_ci': lower_ci_matthews,
        'matthews_test_upper_ci': upper_ci_matthews,
        'accuracy_test': median_accuracy,
        'accuracy_test_lower_ci': lower_ci_accuracy,
        'accuracy_test_upper_ci': upper_ci_accuracy,
        'precision_test': median_precision,
        'precision_test_lower_ci': lower_ci_precision,
        'precision_test_upper_ci': upper_ci_precision,
        'recall_test': median_recall,
        'recall_test_lower_ci': lower_ci_recall,
        'recall_test_upper_ci': upper_ci_recall,
        'specificity_test': median_specificity,
        'specificity_test_lower_ci': lower_ci_specificity,
        'specificity_test_upper_ci': upper_ci_specificity,
        'neg_pred_value_test': median_neg_pred_value,
        'neg_pred_value_test_lower_ci': lower_ci_neg_pred_value,
        'neg_pred_value_test_upper_ci': upper_ci_neg_pred_value,
    }], index=[0])

    return result_df

In [ ]:
def train_and_eval_log_reg(train_data_path, test_data_path, selected_features):
    scenarios = ch.load(path.join(train_data_path))
    models = []
    roc_auc_vals = []
    for scenario in scenarios:
        X_train, X_val, y_train, y_val = scenario
            
        clf = LogisticRegression(random_state=0).fit(X_train[:,0,selected_features], y_train)
        pred_y_val = clf.predict_proba(X_val[:,0,selected_features])[:,1]
        roc_auc_val = roc_auc_score(y_val, pred_y_val)
        models.append(clf)
        roc_auc_vals.append(roc_auc_val)
        
    # select model with highest roc_auc
    best_model_idx = np.argmax(roc_auc_vals)
    print(roc_auc_vals)
    best_model = models[best_model_idx]
    # evaluate best model
    test_data = ch.load(path.join(test_data_path))
    X_test, y_test = test_data
    X_test = X_test[:,0,selected_features]
    result_df = evaluate_model(best_model, X_test, y_test)
       
    return result_df, best_model

In [ ]:
result_df_3m_mrs02, model_3m_mrs02 = train_and_eval_log_reg(train_data_path_3m_mrs02, test_data_path_3m_mrs02, selected_features_3m_mrs02_idx)
result_df_3m_mrs02['outcome'] = '3m_mrs02'

In [ ]:
result_df_3m_mrs02

In [ ]:
result_df_3m_mrs02, model_3m_mrs02 = train_and_eval_log_reg(train_data_path_3m_mrs02, test_data_path_3m_mrs02, selected_features_3m_mrs02_idx)
result_df_3m_mrs02['outcome'] = '3m_mrs02'

result_df_3m_death, model_3m_death = train_and_eval_log_reg(train_data_path_3m_death, test_data_path_3m_death, selected_features_3m_death_idx)
result_df_3m_death['outcome'] = '3m_death'

result_df_hospital_death, model_hospital_death = train_and_eval_log_reg(train_data_path_hospital_death, test_data_path_hospital_death, selected_features_hospital_death_idx)
result_df_hospital_death['outcome'] = 'hospital_death'

In [ ]:
concatenated_df = pd.concat([result_df_3m_mrs02, result_df_3m_death, result_df_hospital_death])
concatenated_df['model'] = 'log_reg'

In [ ]:
concatenated_df

## External validation

In [ ]:
external_features_path = '/Users/jk1/temp/opsum_mimic/preprocessing/mimic_prepro_25012023_232713/preprocessed_features_25012023_232713.csv'
external_labels_path = '/Users/jk1/temp/opsum_mimic/preprocessing/mimic_prepro_25012023_232713/preprocessed_outcomes_25012023_232713.csv'
outcomes = ['3M Death', 'Death in hospital']

In [ ]:
from prediction.outcome_prediction.data_loading.data_loader import load_external_data

In [ ]:
# evaluate model
test_X_np, test_y_np, test_features_lookup_table = load_external_data(external_features_path, external_labels_path, outcomes[0])
result_df_external_3m_death = evaluate_model(model_3m_death, test_X_np[:,0,selected_features_3m_death_idx], test_y_np)
result_df_external_3m_death['outcome'] = '3m_death'

In [ ]:
test_X_np, test_y_np, test_features_lookup_table = load_external_data(external_features_path, external_labels_path, outcomes[1])
result_df_external_hospital_death = evaluate_model(model_hospital_death, test_X_np[:,0,selected_features_hospital_death_idx], test_y_np)
result_df_external_hospital_death['outcome'] = 'hospital_death'

In [ ]:
concatenated_df_external = pd.concat([result_df_external_3m_death, result_df_external_hospital_death])
concatenated_df_external['model'] = 'log_reg'

In [ ]:
concatenated_df_external

join both ds

In [ ]:
concatenated_df['dataset'] = 'gsu'
concatenated_df_external['dataset'] = 'mimic'

In [ ]:
concatenated_df_all = pd.concat([concatenated_df, concatenated_df_external])

In [ ]:
def preprocess_df(df, model_name, outcome, dataset_name):
    preprocessed_df = pd.DataFrame()
    # report result rounded to 3 decimal places with 95% CI
    preprocessed_df['ROC AUC'] = [f'{round(df["auc_test"].values[0], 3):.3f} ({round(df["auc_test_lower_ci"].values[0], 3):.3f}-{round(df["auc_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Matthew's Coefficient"] = [f'{round(df["matthews_test"].values[0], 3):.3f} ({round(df["matthews_test_lower_ci"].values[0], 3):.3f}-{round(df["matthews_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Accuracy"] = [f'{round(df["accuracy_test"].values[0], 3):.3f} ({round(df["accuracy_test_lower_ci"].values[0], 3):.3f}-{round(df["accuracy_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Precision (positive predictive value)"] = [f'{round(df["precision_test"].values[0], 3):.3f} ({round(df["precision_test_lower_ci"].values[0], 3):.3f}-{round(df["precision_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Recall (Sensitivity)"] = [f'{round(df["recall_test"].values[0], 3):.3f} ({round(df["recall_test_lower_ci"].values[0], 3):.3f}-{round(df["recall_test_upper_ci"].values[0], 3):.3f})']
    if 'specificity_test' in df.columns:
        preprocessed_df["Specificity"] = [f'{round(df["specificity_test"].values[0], 3):.3f} ({round(df["specificity_test_lower_ci"].values[0], 3):.3f}-{round(df["specificity_test_upper_ci"].values[0], 3):.3f})']
    else:
        preprocessed_df["Specificity"] = [f'NA']

    preprocessed_df['Model'] = [model_name]
    preprocessed_df['Outcome'] = [outcome]
    preprocessed_df['Dataset'] = [dataset_name]

    return preprocessed_df

In [ ]:
preprocessed_df_all = pd.concat([preprocess_df(concatenated_df[concatenated_df['outcome'] == '3m_mrs02'], 'Logistic Regression', '3M mRS 0-2', 'GSU'),
                                    preprocess_df(concatenated_df[concatenated_df['outcome'] == '3m_death'], 'Logistic Regression', '3M Death', 'GSU'),
                                    preprocess_df(concatenated_df[concatenated_df['outcome'] == 'hospital_death'], 'Logistic Regression', 'Death in hospital', 'GSU'),
                                    preprocess_df(concatenated_df_external[concatenated_df_external['outcome'] == '3m_death'], 'Logistic Regression', '3M Death', 'MIMIC'),
                                    preprocess_df(concatenated_df_external[concatenated_df_external['outcome'] == 'hospital_death'], 'Logistic Regression', 'Death in hospital', 'MIMIC')])

In [ ]:
preprocessed_df_all

In [ ]:
preprocessed_df_all.to_excel('/Users/jk1/Downloads/log_reg_results_simplified_vars.xlsx', index=False)